In [1]:
from bs4 import BeautifulSoup
import requests
# import time, os
import re
# import urllib
# import json
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import numpy as np
import itertools

## Setup

In [8]:
multipliers = dict(default = 1, no_schneider = 2, no_trick = 3, dob = 2, doubler = 2, blitz = 2, crack = 2, crack_back = 2, rb_crack = 4, base = 1, picker = 2)

In [9]:
mult = multipliers

In [3]:
hearts = [str(i).replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A') + 'H' for i in range(7,15)]
clubs = [str(i).replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A')  + 'D' for i in range(7,15)]
spades = [str(i).replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A')  + 'S' for i in range(7,15)]
diamonds = [str(i).replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A')  + 'D' for i in range(7,15)]


In [4]:
allcards = hearts + clubs + spades + diamonds
values = [0, 0, 0, 10, 2, 3, 4, 11, 0, 0, 0, 10, 2, 3, 4, 11, 0, 0, 0, 10, 2, 3, 4, 11, 0, 0, 0, 10, 2, 3, 4, 11]

In [5]:
pointvalues = dict(zip(allcards, values))

In [18]:
picker = 2
norm = 1
ns = 2

In [27]:
picker * ns * mult['blitz'] * mult['doubler'] * mult['crack'] * mult['crack_back']

64

In [28]:
picker * ns * mult['dob'] * mult['blitz'] * mult['doubler'] * mult['crack'] * mult['crack_back']

128

## Scrape

In [6]:
site = 'https://get61.com/hand_detail?handid=2268734'
page = requests.get(site).text
soup = BeautifulSoup(page, 'html5lib')

In [7]:
cardre = re.compile('var play11')
cards = soup.find(text=cardre)
raw_rows = cards.split(';')

b = raw_rows[0].split(' ')

In [8]:
hand_stats = [i.split(' ') for i in raw_rows]
hand_stats[1]

[&#39;\nvar&#39;, &#39;play12&#39;, &#39;=&#39;, &#39;&quot;14C&quot;&#39;]

In [9]:
playnum = [hand_stats[i][1] for i in range(30)]
plays = [hand_stats[i][3].replace('"', '').replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A') for i in range(30)]

In [10]:
leads = [int(hand_stats[i][3].replace('"', '')) for i in range(30, 36)]
scores = [int(hand_stats[i][3].replace('"', '')) for i in range(41, 46)]
bury = [hand_stats[i][3].replace('"', '').replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A') for i in range(46,48)]
blind = [hand_stats[i][3].replace('"', '').replace('11', 'J').replace('12', 'Q').replace('13', 'K').replace('14', 'A') for i in range(48,50)]

In [11]:
# get names
nameslist = [row.split('=') for row in raw_rows[36:41]]
names = [nameslist[row][1].strip().replace('"', '') for row in range(5)]


In [12]:
namescol = []
for i in names:
    namescol.extend([i] * 6)

In [13]:
# playnum, plays, names, leads, scores, bury, blind
df = pd.DataFrame(playnum)
df.columns = ['playnum']
df['plays'] = plays
df['name'] = namescol

In [14]:
n = [1, 2, 3, 4, 5, 6]
trickloop = n* 5
#tricknums = [[str(i)] * 6 for i in range(1,7)]
#tricknums = list(itertools.chain.from_iterable(tricknums))
trickydic = dict(zip(playnum, trickloop))
df['trick'] = df['playnum'].map(trickydic)

In [15]:
positions = [1, 2, 3, 4, 5]
posdic = dict(zip(names, positions))
df['position'] = df['name'].map(posdic)

In [16]:
leadadj = [i + 1 for i in leads]

leadcond = [
    (df['trick'] == 1),
    (df['trick'] == 2),
    (df['trick'] == 3),
    (df['trick'] == 4),
    (df['trick'] == 5),
    (df['trick'] == 6)
]
values = [leadadj]
df['leader'] = np.select(leadcond, leadadj)


In [46]:
handpos1 = (df['name'] == names[0])
p = df[hands]
list(p['plays'])

[&#39;10H&#39;, &#39;AC&#39;, &#39;QH&#39;, &#39;KC&#39;, &#39;9S&#39;, &#39;AH&#39;]

In [64]:
handpos1 = list(df[df['name'] == names[0]]['plays'])
handpos2 = list(df[df['name'] == names[1]]['plays'])
handpos3 = list(df[df['name'] == names[2]]['plays'])
handpos4 = list(df[df['name'] == names[3]]['plays'])
handpos5 = list(df[df['name'] == names[4]]['plays'])

In [57]:
print(handpos1, names[0])
print(handpos2, names[1])
print(handpos3, names[2])
print(handpos4, names[3])
print(handpos5, names[4])

[&#39;10H&#39;, &#39;AC&#39;, &#39;QH&#39;, &#39;KC&#39;, &#39;9S&#39;, &#39;AH&#39;] Bert Olson
[&#39;7H&#39;, &#39;8H&#39;, &#39;JC&#39;, &#39;10S&#39;, &#39;QC&#39;, &#39;8S&#39;] Woller
[&#39;9H&#39;, &#39;7C&#39;, &#39;JD&#39;, &#39;7D&#39;, &#39;KD&#39;, &#39;10D&#39;] Best Looking Guy in the WVC
[&#39;QD&#39;, &#39;8C&#39;, &#39;QS&#39;, &#39;9C&#39;, &#39;7S&#39;, &#39;KS&#39;] 3SheepsToTheWind
[&#39;KH&#39;, &#39;AD&#39;, &#39;8D&#39;, &#39;JS&#39;, &#39;9D&#39;, &#39;JH&#39;] KingCrab


In [59]:
handposfive = df[df['name'] == names[4]]['plays']
handposfive

24    KH
25    AD
26    8D
27    JS
28    9D
29    JH
Name: plays, dtype: object

In [58]:
scores

[2, 2, -2, -4, 2]

In [31]:
ordereddf = df.sort_values(by=['trick', 'position'])
#df.sort_index()
#df.sort_values(by='playnum')
ordereddf

,playnum,plays,name,trick,position,leader
0,play11,10H,Bert Olson,1,1,5
6,play21,7H,Woller,1,2,5
12,play31,9H,Best Looking Guy in the WVC,1,3,5
18,play41,QD,3SheepsToTheWind,1,4,5
24,play51,KH,KingCrab,1,5,5
1,play12,AC,Bert Olson,2,1,4
7,play22,8H,Woller,2,2,4
13,play32,7C,Best Looking Guy in the WVC,2,3,4
19,play42,8C,3SheepsToTheWind,2,4,4
25,play52,AD,KingCrab,2,5,4


In [35]:
gr = ordereddf.groupby(['trick']).reset_index()

AttributeError: &#39;DataFrameGroupBy&#39; object has no attribute &#39;reset_index&#39;

In [34]:
gr

,trick,playnum,plays,name,position,leader
0,1,5,5,5,5,5
1,2,5,5,5,5,5
2,3,5,5,5,5,5
3,4,5,5,5,5,5
4,5,5,5,5,5,5
5,6,5,5,5,5,5


In [66]:
allhands = [handpos1] + [handpos2] + [handpos3] + [handpos4] + [handpos5]

allhands

[[&#39;10H&#39;, &#39;AC&#39;, &#39;QH&#39;, &#39;KC&#39;, &#39;9S&#39;, &#39;AH&#39;],
 [&#39;7H&#39;, &#39;8H&#39;, &#39;JC&#39;, &#39;10S&#39;, &#39;QC&#39;, &#39;8S&#39;],
 [&#39;9H&#39;, &#39;7C&#39;, &#39;JD&#39;, &#39;7D&#39;, &#39;KD&#39;, &#39;10D&#39;],
 [&#39;QD&#39;, &#39;8C&#39;, &#39;QS&#39;, &#39;9C&#39;, &#39;7S&#39;, &#39;KS&#39;],
 [&#39;KH&#39;, &#39;AD&#39;, &#39;8D&#39;, &#39;JS&#39;, &#39;9D&#39;, &#39;JH&#39;]]

In [67]:
table = pd.DataFrame(allhands, columns =[1, 2, 3, 4, 5, 6])

In [68]:
table

,1,2,3,4,5,6
0,10H,AC,QH,KC,9S,AH
1,7H,8H,JC,10S,QC,8S
2,9H,7C,JD,7D,KD,10D
3,QD,8C,QS,9C,7S,KS
4,KH,AD,8D,JS,9D,JH


In [70]:
table['points'] = scores

In [71]:
table

,1,2,3,4,5,6,points
0,10H,AC,QH,KC,9S,AH,2
1,7H,8H,JC,10S,QC,8S,2
2,9H,7C,JD,7D,KD,10D,-2
3,QD,8C,QS,9C,7S,KS,-4
4,KH,AD,8D,JS,9D,JH,2
